# Audio Recorder

In [2]:
import pyaudio
import math
import struct
import wave
import time
import os
import subprocess

## Select music app and persona

In [7]:
# Personas = ["ConnectedCar", "FashionStyle", "Plain"]
# musicApps = ["Spotify", "Amazon", "Pandora"]
Persona = "ConnectedCar"
music_app = "Amazon"

## Make directories

In [11]:
base_dir = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Audio_ads_0326"

Traffic_dir = os.path.join(base_dir, "Traffic", Persona, music_app)
if not os.path.exists(Traffic_dir):
    os.makedirs(Traffic_dir)
    
Recording_dir = os.path.join(base_dir, "Audio_Records", Persona, music_app)
if not os.path.exists(Recording_dir):
    os.makedirs(Recording_dir)

# Record Audio

In [12]:
Threshold = 10
SHORT_NORMALIZE = (1.0/32768.0)
chunk = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
swidth = 2

TIMEOUT_LENGTH = 20

#f_name_directory = r'/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/data/Audio_ads/Audio_Records/ConnectedCar/Pandora//'
f_name_directory = r"{}".format(Recording_dir)

class Recorder:

    @staticmethod
    def rms(frame):
        count = len(frame) / swidth
        format = "%dh" % (count)
        shorts = struct.unpack(format, frame)

        sum_squares = 0.0
        for sample in shorts:
            n = sample * SHORT_NORMALIZE
            sum_squares += n * n
        rms = math.pow(sum_squares / count, 0.5)

        return rms * 1000

    def __init__(self):
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=FORMAT,
                                  channels=CHANNELS,
                                  rate=RATE,
                                  input=True,
                                  output=True,
                                  frames_per_buffer=chunk)

    def record(self):
        print('Noise detected, recording beginning')
        rec = []
        current = time.time()
        end = time.time() + TIMEOUT_LENGTH

        while current <= end:

            data = self.stream.read(chunk)
            if self.rms(data) >= Threshold: end = time.time() + TIMEOUT_LENGTH

            current = time.time()
            rec.append(data)
        self.write(b''.join(rec))

    def write(self, recording):
        n_files = len(os.listdir(f_name_directory))

        filename = os.path.join(f_name_directory, '{}.wav'.format(n_files))

        wf = wave.open(filename, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(self.p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(recording)
        wf.close()
        #print('Written to file: {}'.format(filename))
        print('Returning to listening')



    def listen(self):
        print('Listening beginning')
        while True:
            input = self.stream.read(chunk)
            rms_val = self.rms(input)
            if rms_val > Threshold:
                self.record()

# Capture Traffic

In [18]:
out = os.path.join(Traffic_dir, music_app +".pcap")
command = f"tcpdump -i wlo1 -vvv -w {out}"
p = subprocess.Popen(command, shell=True)

tcpdump: listening on wlo1, link-type EN10MB (Ethernet), capture size 262144 bytes


# Call Skill

In [19]:
record = Recorder()
record.listen()

ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.USB-Audio.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM hdmi
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.USB-Audio.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory


Listening beginning


Noise detected, recording beginning


tcpdump: pcap_loop: The interface went down
0 packets captured
0 packets received by filter
0 packets dropped by kernel


Returning to listening
Noise detected, recording beginning


KeyboardInterrupt: 

In [20]:
command = f"pkill -f tcpdump"
subprocess.Popen(command, shell=True)

<Popen: returncode: None args: 'pkill -f tcpdump'>

# Speech to Text

In [20]:
# importing libraries
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    print(whole_text)

file = "/home/c2/alexa/source/voice-assistant-central/NetworkAnalysis/Audio_Ad/records/0.wav"
get_large_audio_transcription(file)

/home/c2/anaconda3/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


audio-chunks/chunk1.wav : From pandora. 
Error: 
Error: 
Error: 
audio-chunks/chunk5.wav : Broken heart. 
audio-chunks/chunk6.wav : I'm still fixing the crack. 
Error: 
audio-chunks/chunk8.wav : Generated. 
Error: 
Error: 
Error: 
audio-chunks/chunk12.wav : Please. 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
audio-chunks/chunk29.wav : And the pills.. 
audio-chunks/chunk30.wav : Haven't caught up with. 
audio-chunks/chunk31.wav : Underworld. 
audio-chunks/chunk32.wav : Today's hits radio from pandora. 
Error: 
Error: 
Error: 
Error: 
audio-chunks/chunk37.wav : Nearby hookah. 
Error: 
audio-chunks/chunk39.wav : I want someone to love me i need someone. 
Error: 
Error: 
Error: 
audio-chunks/chunk43.wav : Uk. 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
Error: 
audio-chunks/chunk55.wav : Love the mocha siri too bad my budget doesn't sarah don't give it up state farm as o

In [19]:
pip install pydub

  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
Note: you may need to restart the kernel to use updated packages.
